<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

参考：[Kaggle事始め](https://qiita.com/taka4sato/items/802c494fdebeaa7f43b7)

In [97]:
import pandas as pd
import csv as csv
from sklearn.ensemble import RandomForestClassifier

In [98]:
# Load training data
train_df = pd.read_csv("train.csv", header=0)

# Convert "Sex" to be a dummy variable (female = 0, Male = 1)
train_df["Gender"] = train_df["Sex"].map({"female": 0, "male": 1}).astype(int)
train_df.head(3)

# Convert "Embarked" to be a dummy variable (S = 0,C = 1, Q = 2)
train_df["Embarked"] = train_df["Embarked"].dropna().map({"S": 0, "C": 1, "Q": 2}).astype(int)
train_df.head(3)

# Complement the missing values of "Age" column with average of "Age"
median_age = train_df["Age"].dropna().median()
if len(train_df.Age[train_df.Age.isnull()]) > 0:
    train_df.loc[(train_df.Age.isnull()), "Age"] = median_age
    
# Complement the missing values of "Embarked" column with average of "Embarked"
median_emb = train_df["Embarked"].dropna().median()
if len(train_df.Embarked[train_df.Embarked.isnull()]) > 0:
    train_df.loc[(train_df.Embarked.isnull()), "Embarked"] = median_emb
    
# remove un-used columns
train_df = train_df.drop(["Name", "Ticket", "Sex", "Cabin", "PassengerId"], axis=1)
train_df.head(3)

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,Gender
0,0,3,22.0,1,0,7.2500,0.0,1
1,1,1,38.0,1,0,71.2833,1.0,0
2,1,3,26.0,0,0,7.9250,0.0,0


In [99]:
# Load test data, Convert "Sex" to be a dummy variable
test_df = pd.read_csv("test.csv", header=0)
test_df["Gender"] = test_df["Sex"].map({"female": 0, "male": 1}).astype(int)

# Convert "Embarked" to be a dummy variable (S = 0,C = 1, Q = 2)
test_df["Embarked"] = test_df["Embarked"].dropna().map({"S": 0, "C": 1, "Q": 2}).astype(int)
test_df.head(3)

# Complement the missing values of "Age" column with average of "Age"
median_age = test_df["Age"].dropna().median()
if len(test_df.Age[test_df.Age.isnull()]) > 0:
    test_df.loc[(test_df.Age.isnull()), "Age"] = median_age
    
# Complement the missing values of "Embarked" column with average of "Embarked"
median_emb = test_df["Embarked"].dropna().median()
if len(test_df.Embarked[test_df.Embarked.isnull()]) > 0:
    test_df.loc[(test_df.Embarked.isnull()), "Embarked"] = median_emb
    
# Complement the missing values of "Fare" column with average of "Fare"
median_fare = test_df["Fare"].dropna().median()
if len(test_df.Fare[test_df.Fare.isnull()]) > 0:
    test_df.loc[(test_df.Fare.isnull()), "Fare"] = median_fare

# Copy test data's "PassengerId" column, and remove un-used columns
ids = test_df["PassengerId"].values
test_df = test_df.drop(["Name", "Ticket", "Sex", "Cabin", "PassengerId"], axis=1)
test_df.head(3)

,Pclass,Age,SibSp,Parch,Fare,Embarked,Gender
0,3,34.5,0,0,7.8292,2,1
1,3,47.0,1,0,7.0000,0,0
2,2,62.0,0,0,9.6875,2,1


In [100]:
train_x = train_df.values[:,1:]
train_y = train_df.values[:,0]

In [101]:
print("train_x:",train_x.shape,"train_y:",train_y.shape)

train_x: (891, 7) train_y: (891,)


In [102]:
from tensorflow.python.keras.utils import to_categorical

In [103]:
train_y = to_categorical(train_y)

In [104]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout

In [111]:
def current_best_model():
    model.add(
        Dense(
            units=128, 
            input_shape=(7,),
            activation='relu'
            )
        )

    model.add(Dropout(0.25))

    model.add(
        Dense(
            units=64, 
            activation='relu'
        )
    )

    model.add(Dropout(0.25))

    model.add(
        Dense(
            units=32, 
            activation='relu'
        )
    )

    model.add(
        Dense(
            units=2, 
            activation='softmax'
        )
    )

In [150]:
def latest_model():
    model.add(
        Dense(
            units=14, 
            input_shape=(7,),
            activation='relu'
            )
    )

    model.add(
        Dense(
            units=4, 
            activation='relu'
        )
    )

    model.add(
        Dense(
            units=2, 
            activation='softmax'
        )
    )

In [151]:
model = Sequential()

In [152]:
#current_best_model()
latest_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_129 (Dense)            (None, 14)                112       
_________________________________________________________________
dense_130 (Dense)            (None, 4)                 60        
_________________________________________________________________
dense_131 (Dense)            (None, 2)                 10        
Total params: 182
Trainable params: 182
Non-trainable params: 0
_________________________________________________________________


In [153]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_adam = model.fit(
    train_x,
    train_y,
    batch_size=32,
    epochs=500,
    validation_split=0.2)

Train on 712 samples, validate on 179 samples
Epoch 1/500
712/712 [==============================] - 3s 4ms/step - loss: 4.2142 - acc: 0.6124 - val_loss: 2.6658 - val_acc: 0.6536
Epoch 2/500
712/712 [==============================] - 0s 137us/step - loss: 1.7066 - acc: 0.6278 - val_loss: 0.7402 - val_acc: 0.7151
Epoch 3/500
712/712 [==============================] - 0s 129us/step - loss: 0.8763 - acc: 0.6728 - val_loss: 0.5936 - val_acc: 0.7095
Epoch 4/500
712/712 [==============================] - 0s 116us/step - loss: 0.6991 - acc: 0.6404 - val_loss: 0.5725 - val_acc: 0.6592
Epoch 5/500
712/712 [==============================] - 0s 114us/step - loss: 0.6566 - acc: 0.6110 - val_loss: 0.5741 - val_acc: 0.6145
Epoch 6/500
712/712 [==============================] - 0s 114us/step - loss: 0.6447 - acc: 0.6011 - val_loss: 0.5677 - val_acc: 0.6369
Epoch 7/500
712/712 [==============================] - 0s 115us/step - loss: 0.6385 - acc: 0.5969 - val_loss: 0.5669 - val_acc: 0.6313
Epoch 8/500

Epoch 61/500
712/712 [==============================] - 0s 199us/step - loss: 0.5332 - acc: 0.7486 - val_loss: 0.4575 - val_acc: 0.7989
Epoch 62/500
712/712 [==============================] - 0s 159us/step - loss: 0.5327 - acc: 0.7388 - val_loss: 0.4573 - val_acc: 0.8101
Epoch 63/500
712/712 [==============================] - 0s 100us/step - loss: 0.5277 - acc: 0.7416 - val_loss: 0.4586 - val_acc: 0.7933
Epoch 64/500
712/712 [==============================] - 0s 219us/step - loss: 0.5255 - acc: 0.7500 - val_loss: 0.4641 - val_acc: 0.7821
Epoch 65/500
712/712 [==============================] - 0s 122us/step - loss: 0.5252 - acc: 0.7472 - val_loss: 0.4590 - val_acc: 0.8101
Epoch 66/500
712/712 [==============================] - 0s 98us/step - loss: 0.5248 - acc: 0.7626 - val_loss: 0.4562 - val_acc: 0.8045
Epoch 67/500
712/712 [==============================] - 0s 107us/step - loss: 0.5290 - acc: 0.7472 - val_loss: 0.4637 - val_acc: 0.7821
Epoch 68/500
712/712 [===========================

712/712 [==============================] - 0s 265us/step - loss: 0.5117 - acc: 0.7584 - val_loss: 0.4619 - val_acc: 0.7933
Epoch 122/500
712/712 [==============================] - 0s 146us/step - loss: 0.5103 - acc: 0.7654 - val_loss: 0.4734 - val_acc: 0.7877
Epoch 123/500
712/712 [==============================] - 0s 146us/step - loss: 0.5179 - acc: 0.7472 - val_loss: 0.4555 - val_acc: 0.7933
Epoch 124/500
712/712 [==============================] - 0s 146us/step - loss: 0.5168 - acc: 0.7542 - val_loss: 0.4498 - val_acc: 0.8268
Epoch 125/500
712/712 [==============================] - 0s 214us/step - loss: 0.5176 - acc: 0.7584 - val_loss: 0.4500 - val_acc: 0.7877
Epoch 126/500
712/712 [==============================] - 0s 106us/step - loss: 0.5090 - acc: 0.7640 - val_loss: 0.4477 - val_acc: 0.8045
Epoch 127/500
712/712 [==============================] - 0s 104us/step - loss: 0.5141 - acc: 0.7612 - val_loss: 0.4450 - val_acc: 0.8045
Epoch 128/500
712/712 [==============================] 

KeyboardInterrupt: 

In [108]:
res = model.predict(test_df.values)

In [109]:
output = [0 for i in range(len(res))]
for i in range(len(res)):
    if res[i][0] >= 0.5:
        output[i] = 0
    else:
        output[i] = 1

In [110]:
submit_file = open("titanic_submit.csv", "w")
file_object = csv.writer(submit_file)
file_object.writerow(["PassengerId", "Survived"])
file_object.writerows(zip(ids, output))
submit_file.close()